#ПОДКЛЮЧЕНИЕ GOOGLE DRIVE И НАСТРОЙКА СРЕДЫ


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q tensorflow opencv-python scikit-learn scikit-image pandas numpy matplotlib tqdm pydicom medpy
!pip install -q ultralytics segment-anything timm
!pip install -q git+https://github.com/facebookresearch/segment-anything.git

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import pydicom
from medpy.metric import dc, jc
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import torch
from ultralytics import YOLO
from segment_anything import sam_model_registry
import yaml
import shutil

# Настройки
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Проверка GPU
print(f"TensorFlow version: {tf.__version__}")
print(f"Num GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")

#ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ



In [ ]:
BASE_PATH = '/content/drive/MyDrive/CHAOS_Train_Sets/Train_Sets/CT'

def load_dicom_file(file_path):
    """Загружает DICOM файл и возвращает нормализованное изображение"""
    ds = pydicom.dcmread(file_path)
    img = ds.pixel_array.astype(np.float32)
    # Нормализация в диапазон [0, 1]
    img_min = img.min()
    img_max = img.max()
    if img_max > img_min:
        img = (img - img_min) / (img_max - img_min)
    else:
        img = img - img_min
    return img, ds

def get_data(base_path, max_patients=1):
    """Собирает пути к DICOM файлам и маскам из датасета CHAOS"""
    X_filenames = []
    y_filenames = []

    # Получаем список всех папок с пациентами
    all_patient_dirs = sorted([d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))])
    # Ограничиваем количество пациентов
    patient_dirs = all_patient_dirs[:max_patients]
    print(f"Берем {len(patient_dirs)} пациентов из {len(all_patient_dirs)} доступных")

    # Проходим по отфильтрованным папкам с номерами пациентов
    for patient_dir in patient_dirs:
        patient_path = os.path.join(base_path, patient_dir)

        # Пути к DICOM файлам и маскам
        dicom_path = os.path.join(patient_path, 'DICOM_anon')
        ground_path = os.path.join(patient_path, 'Ground')

        if os.path.exists(dicom_path):
            for filename in sorted(os.listdir(dicom_path)):
                if filename.lower().endswith('.dcm'):
                    X_filenames.append(os.path.join(dicom_path, filename))

        if os.path.exists(ground_path):
            for filename in sorted(os.listdir(ground_path)):
                if filename.lower().endswith('.png'):
                    y_filenames.append(os.path.join(ground_path, filename))

    print(f"Найдено {len(X_filenames)} DICOM файлов и {len(y_filenames)} масок")
    X_filenames = sorted(X_filenames)
    y_filenames = sorted(y_filenames)
    return X_filenames, y_filenames

def buffer_imgs(filenames, is_dicom, folder='buffer'):
    """Конвертирует файлы в формат, удобный для быстрой загрузки"""
    if not os.path.exists(folder):
        os.makedirs(folder)

    processed_files = []
    for i, filename in enumerate(tqdm(filenames, desc=f'Обработка {"DICOM" if is_dicom else "масок"}')):
        base_name = os.path.basename(filename)
        name_without_ext = os.path.splitext(base_name)[0]
        safe_name = f"{i:06d}_{name_without_ext}"

        if is_dicom:
            img, _ = load_dicom_file(filename)
            output_path = os.path.join(folder, f"{safe_name}.tiff")
            img_16bit = (img * 65535).astype(np.uint16)
            Image.fromarray(img_16bit).save(output_path, compression='tiff_lzw')
        else:
            img = np.array(Image.open(filename))
            output_path = os.path.join(folder, f"{safe_name}.png")
            img_8bit = (img * 255).astype(np.uint8)
            Image.fromarray(img_8bit).save(output_path)

        processed_files.append(output_path)

    return pd.DataFrame(processed_files, columns=['filename'])

# Загрузка данных
print("Загрузка данных")
X_filenames, y_filenames = get_data(BASE_PATH, max_patients=5)

# Создаем буферные файлы
print("Создание буферных файлов")
X_df = buffer_imgs(X_filenames, True, 'buffer/images')
y_df = buffer_imgs(y_filenames, False, 'buffer/masks')
print(f"Создано {len(X_df)} изображений и {len(y_df)} масок")

#МЕТРИКИ И КОНФИГУРАЦИИ

In [ ]:
def dice_coefficient(y_true, y_pred, smooth=1e-7):
    """Вычисляет Dice коэффициент между двумя тензорами"""
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred > 0.5, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)
    return (2. * intersection + smooth) / (union + smooth)

def jaccard_index(y_true, y_pred, smooth=1e-7):
    """Вычисляет Jaccard индекс (IoU) между двумя тензорами"""
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred > 0.5, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def precision_metric(y_true, y_pred, smooth=1e-7):
    """Вычисляет Precision метрику"""
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred > 0.5, [-1]), tf.float32)
    true_positives = tf.reduce_sum(y_true_f * y_pred_f)
    predicted_positives = tf.reduce_sum(y_pred_f)
    return (true_positives + smooth) / (predicted_positives + smooth)

def recall_metric(y_true, y_pred, smooth=1e-7):
    """Вычисляет Recall метрику"""
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred > 0.5, [-1]), tf.float32)
    true_positives = tf.reduce_sum(y_true_f * y_pred_f)
    actual_positives = tf.reduce_sum(y_true_f)
    return (true_positives + smooth) / (actual_positives + smooth)

def dice_loss(y_true, y_pred):
    """Loss функция на основе Dice коэффициента"""
    return 1 - dice_coefficient(y_true, y_pred)

def combined_loss(y_true, y_pred, alpha=0.5):
    """Комбинированная loss функция (Dice + BCE)"""
    dice = dice_loss(y_true, y_pred)
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    return alpha * dice + (1 - alpha) * bce

# Базовые конфигурации
BASE_CONFIG = {
    'batch_size': 2,
    'img_size': 256,
    'epochs': 10,
    'learning_rate': 0.001,
    'patience': 10,
    'min_lr': 1e-6,
    'augmentation': {
        'rotation_range': 10,
        'width_shift_range': 0.1,
        'height_shift_range': 0.1,
        'horizontal_flip': True,
        'zoom_range': 0.1,
        'fill_mode': 'reflect'
    }
}

def create_data_generators(X_df, y_df, config, validation_split=0.2):
    """Создает генераторы данных для обучения и валидации"""
    # Разделение данных
    X_train, X_val, y_train, y_val = train_test_split(
        X_df, y_df, test_size=validation_split, random_state=SEED, shuffle=True
    )

    # Генераторы для обучения
    train_img_datagen = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization=True,
        **config['augmentation']
    )
    train_mask_datagen = ImageDataGenerator(
        rescale=1./255,
        **config['augmentation']
    )

    # Генераторы для валидации
    val_img_datagen = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization=True
    )
    val_mask_datagen = ImageDataGenerator(rescale=1./255)

    # Параметры генераторов
    gen_params = {
        'x_col': 'filename',
        'class_mode': None,
        'color_mode': 'grayscale',
        'target_size': (config['img_size'], config['img_size']),
        'batch_size': config['batch_size'],
        'seed': SEED,
        'shuffle': True
    }
    val_gen_params = gen_params.copy()
    val_gen_params['shuffle'] = False

    # Создание генераторов
    train_img_gen = train_img_datagen.flow_from_dataframe(X_train, **gen_params)
    train_mask_gen = train_mask_datagen.flow_from_dataframe(y_train, **gen_params)
    val_img_gen = val_img_datagen.flow_from_dataframe(X_val, **val_gen_params)
    val_mask_gen = val_mask_datagen.flow_from_dataframe(y_val, **val_gen_params)

    # Объединение генераторов
    def combine_generators(img_gen, mask_gen):
        while True:
            imgs = next(img_gen)
            masks = next(mask_gen)
            yield imgs, masks.astype(np.float32)

    train_generator = combine_generators(train_img_gen, train_mask_gen)
    val_generator = combine_generators(val_img_gen, val_mask_gen)

    steps_per_epoch = len(X_train) // config['batch_size']
    validation_steps = len(X_val) // config['batch_size']

    return train_generator, val_generator, steps_per_epoch, validation_steps

#ФАБРИКА МОДЕЛЕЙ

In [ ]:
class ModelFactory:
    """Фабрика для создания различных моделей сегментации"""

    @staticmethod
    def create_unet(input_shape, neurons=16, name='unet'):
        """Создает U-Net модель с заданными параметрами"""
        inputs = Input(shape=input_shape)

        # Encoder
        conv1 = Conv2D(neurons*1, (3, 3), activation='relu', padding='same')(inputs)
        conv1 = Conv2D(neurons*1, (3, 3), activation='relu', padding='same')(conv1)
        conv1 = BatchNormalization()(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        pool1 = Dropout(0.25)(pool1)

        conv2 = Conv2D(neurons*2, (3, 3), activation='relu', padding='same')(pool1)
        conv2 = Conv2D(neurons*2, (3, 3), activation='relu', padding='same')(conv2)
        conv2 = BatchNormalization()(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        pool2 = Dropout(0.5)(pool2)

        conv3 = Conv2D(neurons*4, (3, 3), activation='relu', padding='same')(pool2)
        conv3 = Conv2D(neurons*4, (3, 3), activation='relu', padding='same')(conv3)
        conv3 = BatchNormalization()(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        pool3 = Dropout(0.5)(pool3)

        conv4 = Conv2D(neurons*8, (3, 3), activation='relu', padding='same')(pool3)
        conv4 = Conv2D(neurons*8, (3, 3), activation='relu', padding='same')(conv4)
        conv4 = BatchNormalization()(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
        pool4 = Dropout(0.5)(pool4)

        # Bottleneck
        conv5 = Conv2D(neurons*16, (3, 3), activation='relu', padding='same')(pool4)
        conv5 = Conv2D(neurons*16, (3, 3), activation='relu', padding='same')(conv5)

        # Decoder
        up6 = UpSampling2D(size=(2, 2))(conv5)
        up6 = concatenate([up6, conv4], axis=3)
        up6 = Dropout(0.5)(up6)
        conv6 = Conv2D(neurons*8, (3, 3), activation='relu', padding='same')(up6)
        conv6 = Conv2D(neurons*8, (3, 3), activation='relu', padding='same')(conv6)

        up7 = UpSampling2D(size=(2, 2))(conv6)
        up7 = concatenate([up7, conv3], axis=3)
        up7 = Dropout(0.5)(up7)
        conv7 = Conv2D(neurons*4, (3, 3), activation='relu', padding='same')(up7)
        conv7 = Conv2D(neurons*4, (3, 3), activation='relu', padding='same')(conv7)

        up8 = UpSampling2D(size=(2, 2))(conv7)
        up8 = concatenate([up8, conv2], axis=3)
        up8 = Dropout(0.5)(up8)
        conv8 = Conv2D(neurons*2, (3, 3), activation='relu', padding='same')(up8)
        conv8 = Conv2D(neurons*2, (3, 3), activation='relu', padding='same')(conv8)

        up9 = UpSampling2D(size=(2, 2))(conv8)
        up9 = concatenate([up9, conv1], axis=3)
        up9 = Dropout(0.5)(up9)
        conv9 = Conv2D(neurons*1, (3, 3), activation='relu', padding='same')(up9)
        conv9 = Conv2D(neurons*1, (3, 3), activation='relu', padding='same')(conv9)

        # Output layer
        conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

        model = Model(inputs=[inputs], outputs=[conv10], name=name)

        # Компиляция модели
        model.compile(
            optimizer=Adam(learning_rate=BASE_CONFIG['learning_rate']),
            loss=combined_loss,
            metrics=[dice_coefficient, jaccard_index, precision_metric, recall_metric]
        )
        return model

    @staticmethod
    def create_yolo11n_seg():
        """Создает YOLO11n-seg модель"""
        return YOLO('yolo11n-seg.pt')

    @staticmethod
    def create_yolo11s_seg():
        """Создает YOLO11s-seg модель"""
        return YOLO('yolo11s-seg.pt')

    @staticmethod
    def create_sam2_model():
        """Создает SAM 2 модель"""
        # Скачиваем checkpoint
        sam_checkpoint = "/content/sam_vit_b_01ec64.pth"
        if not os.path.exists(sam_checkpoint):
            print("Скачивание SAM checkpoint")
            !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -O {sam_checkpoint}

        # Загружаем модель
        sam_model = sam_model_registry["vit_b"](checkpoint=sam_checkpoint)
        sam_model.train()
        return sam_model

#ПОДГОТОВКА ДАННЫХ ПОД КАЖДУЮ МОДЕЛЬ

In [ ]:
def prepare_unet_data(X_df, y_df, config):
    """Подготавливает данные для U-Net модели"""
    return create_data_generators(X_df, y_df, config)

def prepare_yolo_data(X_df, y_df, output_dir='/content/yolo_dataset'):
    """Подготавливает данные в формате YOLO для сегментации"""
    # Очищаем предыдущие данные
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # Создаем структуру папок
    for split in ['train', 'val']:
        os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)

    # Разделение данных
    X_train, X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=0.2, random_state=SEED)

    def process_split(X_split, y_split, split_name):
        image_paths = []
        label_paths = []
        for i in tqdm(range(len(X_split)), desc=f'Обработка {split_name}'):
            img_path = X_split.iloc[i, 0]
            mask_path = y_split.iloc[i, 0]

            # Загружаем изображение
            img = np.array(Image.open(img_path))
            if img.dtype == np.uint16:
                img = (img.astype(np.float32) / 65535 * 255).astype(np.uint8)

            # Сохраняем изображение
            img_filename = f'image_{i}.jpg'
            img_save_path = os.path.join(output_dir, 'images', split_name, img_filename)
            Image.fromarray(img).save(img_save_path, quality=95)
            image_paths.append(img_save_path)

            # Обрабатываем маску
            mask = np.array(Image.open(mask_path))
            mask_binary = (mask > 128).astype(np.uint8)

            # Находим контуры для сегментации
            contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                contour = max(contours, key=cv2.contourArea)
                img_h, img_w = img.shape[:2]
                normalized_contour = contour.reshape(-1, 2).astype(np.float32) / np.array([img_w, img_h])

                # Сохраняем аннотацию
                label_filename = f'image_{i}.txt'
                label_save_path = os.path.join(output_dir, 'labels', split_name, label_filename)
                with open(label_save_path, 'w') as f:
                    f.write('0 ')  # class_id для печени
                    for point in normalized_contour:
                        f.write(f'{point[0]:.6f} {point[1]:.6f} ')
                label_paths.append(label_save_path)

        return image_paths, label_paths

    # Обрабатываем train и val
    train_imgs, train_labels = process_split(X_train, y_train, 'train')
    val_imgs, val_labels = process_split(X_val, y_val, 'val')

    # Создаем конфигурационный файл
    yolo_config = {
        'path': output_dir,
        'train': 'images/train',
        'val': 'images/val',
        'names': {0: 'liver'},
        'nc': 1
    }
    config_path = os.path.join(output_dir, 'data.yaml')
    with open(config_path, 'w') as f:
        yaml.dump(yolo_config, f)

    return config_path, len(train_imgs), len(val_imgs)

def prepare_sam_data(X_df, y_df, output_dir='/content/sam_dataset'):
    """Подготавливает данные для SAM 2 модели"""
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # Разделение данных
    X_train, X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=0.2, random_state=SEED)

    def process_split(X_split, y_split, split_name):
        images = []
        masks = []
        for i in tqdm(range(len(X_split)), desc=f'Обработка SAM {split_name}'):
            img_path = X_split.iloc[i, 0]
            mask_path = y_split.iloc[i, 0]

            # Загружаем изображение
            img = np.array(Image.open(img_path))
            if img.dtype == np.uint16:
                img = (img.astype(np.float32) / 65535 * 255).astype(np.uint8)

            # Преобразуем в 3 канала для SAM
            if len(img.shape) == 2:
                img = np.stack([img, img, img], axis=-1)

            # Сохраняем изображение
            img_filename = f'image_{i}.jpg'
            img_save_path = os.path.join(output_dir, split_name, img_filename)
            os.makedirs(os.path.dirname(img_save_path), exist_ok=True)
            Image.fromarray(img).save(img_save_path, quality=95)
            images.append(img_save_path)

            # Сохраняем маску
            mask = np.array(Image.open(mask_path))
            mask_filename = f'mask_{i}.png'
            mask_save_path = os.path.join(output_dir, split_name, mask_filename)
            Image.fromarray(mask).save(mask_save_path)
            masks.append(mask_save_path)

        return images, masks

    # Обрабатываем train и val
    train_images, train_masks = process_split(X_train, y_train, 'train')
    val_images, val_masks = process_split(X_val, y_val, 'val')

    return train_images, train_masks, val_images, val_masks

#ШАБЛОНЫ ОБУЧЕНИЯ И ОЦЕНКИ С ВИЗУАЛИЗАЦИЕЙ КОНТУРОВ

In [ ]:
def train_unet_model(model, train_generator, val_generator, steps_per_epoch, validation_steps, config, model_name):
    """Обучает U-Net модель с сохранением лучшей версии"""
    # Коллбэки
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=config['patience'] // 2,
        min_lr=config['min_lr'],
        verbose=1
    )
    early_stop = EarlyStopping(
        monitor='val_dice_coefficient',
        patience=config['patience'],
        mode='max',
        restore_best_weights=True,
        verbose=1
    )
    # checkpoint для лучших весов
    best_checkpoint = ModelCheckpoint(
        f'/content/{model_name}_best.h5',
        monitor='val_dice_coefficient',
        save_best_only=True,
        mode='max',
        verbose=1
    )
    # checkpoint для последних весов
    final_checkpoint = ModelCheckpoint(
        f'/content/{model_name}_final.h5',
        save_best_only=False,
        verbose=0
    )
    callbacks = [reduce_lr, early_stop, best_checkpoint, final_checkpoint]

    print(f"\n{'='*50}")
    print(f"ОБУЧЕНИЕ {model_name.upper()}")
    print(f"{'='*50}")

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        epochs=config['epochs'],
        callbacks=callbacks,
        verbose=1
    )

    print(f"\nЛучшая модель сохранена")
    print(f"Финальная модель сохранена")

    return history

def train_yolo_model(model, data_config, config, model_name):
    """Обучает YOLO модель с сохранением лучшей версии"""
    print(f"\n{'='*50}")
    print(f"ОБУЧЕНИЕ {model_name.upper()}")
    print(f"{'='*50}")

    # Обучение
    results = model.train(
        data=data_config,
        epochs=config['epochs'],
        batch=config['batch_size'],
        imgsz=config['img_size'],
        name=model_name,
        patience=config['patience'],
        device='cpu',
        verbose=True,
        plots=True,
        save=True,  # Всегда сохранять чекпойнты
        save_period=1  # Сохранять каждую эпоху
    )

    # Копируем лучшую модель в основную директорию
    results_dir = f'/content/runs/segment/{model_name}'
    best_model_path = os.path.join(results_dir, 'weights', 'best.pt')
    final_model_path = os.path.join(results_dir, 'weights', 'last.pt')

    if os.path.exists(best_model_path):
        shutil.copy(best_model_path, f'/content/{model_name}_best.pt')
        print(f"Лучшая модель скопирована")

    if os.path.exists(final_model_path):
        shutil.copy(final_model_path, f'/content/{model_name}_final.pt')
        print(f"Финальная модель скопирована")

    return results

def train_sam_model(model, train_data, val_data, config, model_name):
    """Обучает SAM 2 модель с сохранением лучшей версии"""
    # Замораживаем ненужные слои
    for param in model.image_encoder.parameters():
        param.requires_grad = False
    for param in model.prompt_encoder.parameters():
        param.requires_grad = False
    # Обучаем только mask_decoder
    for param in model.mask_decoder.parameters():
        param.requires_grad = True

    print(f"Обучаемые параметры: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

    train_images, train_masks = train_data
    val_images, val_masks = val_data
    TARGET_SIZE = (1024, 1024)
    MASK_SIZE = (256, 256)

    optimizer = torch.optim.Adam(model.mask_decoder.parameters(), lr=config['learning_rate'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

    history = {'train_loss': [], 'val_loss': [], 'train_dice': [], 'val_dice': []}
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    # Отслеживание лучшей метрики
    best_val_dice = 0.0
    best_epoch = 0

    print(f"\n{'='*50}")
    print(f"FINE-TUNING {model_name.upper()}")
    print(f"{'='*50}")

    for epoch in range(config['epochs']):
        # Обучение
        model.train()
        train_loss = 0
        train_dice = 0
        processed_count = 0

        for i in tqdm(range(len(train_images)), desc=f'Epoch {epoch+1}/{config["epochs"]}'):
            img_path = train_images[i]
            mask_path = train_masks[i]

            img = np.array(Image.open(img_path))
            mask = np.array(Image.open(mask_path))

            # Обработка изображения для SAM
            if img.dtype == np.uint16:
                img = (img.astype(np.float32) / 65535 * 255).astype(np.uint8)
            if len(img.shape) == 2:
                img = np.stack([img, img, img], axis=-1)

            # Ресайзинг до размера SAM
            img = cv2.resize(img, TARGET_SIZE, interpolation=cv2.INTER_LINEAR)
            mask = cv2.resize(mask, MASK_SIZE, interpolation=cv2.INTER_NEAREST)

            # Нормализация
            img = img.astype(np.float32) / 255.0
            img_tensor = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)
            mask_tensor = torch.from_numpy(mask).unsqueeze(0).unsqueeze(0).to(device)

            # Forward pass
            with torch.no_grad():
                image_embedding = model.image_encoder(img_tensor)

            # Генерация prompts (без точек/боксов)
            sparse_embeddings, dense_embeddings = model.prompt_encoder(
                points=None,
                boxes=None,
                masks=None,
            )

            # Предсказание масок
            low_res_masks, iou_predictions = model.mask_decoder(
                image_embeddings=image_embedding,
                image_pe=model.prompt_encoder.get_dense_pe(),
                sparse_prompt_embeddings=sparse_embeddings,
                dense_prompt_embeddings=dense_embeddings,
                multimask_output=False,
            )

            # Loss и оптимизация
            loss = torch.nn.BCEWithLogitsLoss()(low_res_masks, (mask_tensor > 128).float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Вычисление Dice
            with torch.no_grad():
                pred_mask = (torch.sigmoid(low_res_masks) > 0.5).float()
                dice = (2 * (pred_mask * (mask_tensor > 128).float()).sum()) / \
                       (pred_mask.sum() + (mask_tensor > 128).float().sum() + 1e-8)
                train_dice += dice.item()

            processed_count += 1

        # Валидация
        model.eval()
        val_loss = 0
        val_dice = 0
        val_count = 0

        with torch.no_grad():
            for i in range(min(10, len(val_images))):
                img_path = val_images[i]
                mask_path = val_masks[i]

                img = np.array(Image.open(img_path))
                mask = np.array(Image.open(mask_path))

                if img.dtype == np.uint16:
                    img = (img.astype(np.float32) / 65535 * 255).astype(np.uint8)
                if len(img.shape) == 2:
                    img = np.stack([img, img, img], axis=-1)

                img = cv2.resize(img, TARGET_SIZE, interpolation=cv2.INTER_LINEAR)
                mask = cv2.resize(mask, MASK_SIZE, interpolation=cv2.INTER_NEAREST)

                img = img.astype(np.float32) / 255.0
                img_tensor = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)
                mask_tensor = torch.from_numpy(mask).unsqueeze(0).unsqueeze(0).to(device)

                image_embedding = model.image_encoder(img_tensor)
                sparse_embeddings, dense_embeddings = model.prompt_encoder(
                    points=None, boxes=None, masks=None
                )

                low_res_masks, _ = model.mask_decoder(
                    image_embeddings=image_embedding,
                    image_pe=model.prompt_encoder.get_dense_pe(),
                    sparse_prompt_embeddings=sparse_embeddings,
                    dense_prompt_embeddings=dense_embeddings,
                    multimask_output=False,
                )

                loss = torch.nn.BCEWithLogitsLoss()(low_res_masks, (mask_tensor > 128).float())
                val_loss += loss.item()

                pred_mask = (torch.sigmoid(low_res_masks) > 0.5).float()
                dice = (2 * (pred_mask * (mask_tensor > 128).float()).sum()) / \
                       (pred_mask.sum() + (mask_tensor > 128).float().sum() + 1e-8)
                val_dice += dice.item()

                val_count += 1

        avg_train_loss = train_loss / len(train_images)
        avg_train_dice = train_dice / len(train_images)
        avg_val_loss = val_loss / min(10, len(val_images))
        avg_val_dice = val_dice / min(10, len(val_images))

        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['train_dice'].append(avg_train_dice)
        history['val_dice'].append(avg_val_dice)

        scheduler.step(avg_val_loss)

        # СОХРАНЕНИЕ ЛУЧШЕЙ МОДЕЛИ
        if avg_val_dice > best_val_dice:
            best_val_dice = avg_val_dice
            best_epoch = epoch + 1

            # Сохраняем лучшую модель
            best_model_path = f'/content/{model_name}_best.pth'
            torch.save({
                'epoch': best_epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_dice': best_val_dice,
                'history': history
            }, best_model_path)

            print(f"Лучшая модель сохранена с Dice={best_val_dice:.4f}")
        else:
            print(f"Лучший результат не изменен")

        print(f'Epoch {epoch+1}/{config["epochs"]}: loss={avg_train_loss:.4f}, val_loss={avg_val_loss:.4f}, dice={avg_train_dice:.4f}, val_dice={avg_val_dice:.4f}')

    # Сохранение финальной модели
    final_model_path = f'/content/{model_name}_final.pth'
    torch.save({
        'epoch': config['epochs'],
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'final_dice': avg_val_dice,
        'history': history
    }, final_model_path)
    print(f"\nФинальная модель сохранена")
    print(f"Лучшая модель сохранена")

    return history

def evaluate_model_with_contours(model, test_data, model_type, model_name, num_samples=10):
    """
    Оценивает модель на тестовых данных с визуализацией контуров
    """
    print(f"\n{'='*30}")
    print(f"ОЦЕНКА {model_name.upper()} С КОНТУРАМИ")
    print(f"{'='*30}")

    dices = []
    ious = []
    precisions = []
    recalls = []

    if model_type == 'unet':
        processed_samples = 0
        for i, (images, masks) in enumerate(test_data):
            if processed_samples >= num_samples:
                break
            preds = model.predict(images, verbose=0)
            for j in range(len(images)):
                if processed_samples >= num_samples:
                    break
                true_mask = (masks[j] > 0.5).astype(np.uint8).squeeze()
                pred_mask = (preds[j] > 0.5).astype(np.uint8).squeeze()

                # Вычисление метрик
                dice = dc(pred_mask, true_mask)
                iou = jc(pred_mask, true_mask)
                intersection = np.logical_and(pred_mask, true_mask).sum()
                precision = intersection / (pred_mask.sum() + 1e-8)
                recall = intersection / (true_mask.sum() + 1e-8)

                dices.append(dice)
                ious.append(iou)
                precisions.append(precision)
                recalls.append(recall)

                # Визуализация первых 3 примеров с контурами
                if processed_samples < 3:
                    # Подготовка контуров
                    true_mask_8uc1 = (true_mask * 255).astype(np.uint8)
                    pred_mask_8uc1 = (pred_mask * 255).astype(np.uint8)

                    contours_true, _ = cv2.findContours(true_mask_8uc1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    contours_pred, _ = cv2.findContours(pred_mask_8uc1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    plt.figure(figsize=(12, 4))
                    plt.subplot(1, 3, 1)
                    plt.imshow(images[j].squeeze(), cmap='gray')
                    plt.title('Original Image')
                    plt.axis('off')

                    plt.subplot(1, 3, 2)
                    plt.imshow(true_mask, cmap='jet')
                    for contour in contours_true:
                        plt.plot(contour[:, 0, 0], contour[:, 0, 1], 'b', linewidth=2)
                    plt.title('Ground Truth')
                    plt.axis('off')

                    plt.subplot(1, 3, 3)
                    plt.imshow(pred_mask, cmap='jet')
                    for contour in contours_pred:
                        plt.plot(contour[:, 0, 0], contour[:, 0, 1], 'r', linewidth=2)
                    plt.title(f'Prediction\nDice: {dice:.3f}')
                    plt.axis('off')

                    plt.suptitle(f'{model_name} - Sample {processed_samples+1}')
                    plt.tight_layout()
                    plt.show()

                processed_samples += 1

    elif model_type == 'yolo':
        results_base_dir = '/content/runs/segment'
        if not os.path.exists(results_base_dir):
            print(f"Директория {results_base_dir} не найдена")
            return extract_yolo_metrics(model_name)

        all_folders = [f for f in os.listdir(results_base_dir)
                      if os.path.isdir(os.path.join(results_base_dir, f)) and 'yolo11' in f]
        if not all_folders:
            print(f"Не найдено папок YOLO в {results_base_dir}")
            return extract_yolo_metrics(model_name)

        all_folders.sort()
        latest_folder = all_folders[-1]
        results_dir = os.path.join(results_base_dir, latest_folder)
        print(f"Используется папка: {latest_folder}")

        # Ищем файлы с результатами
        result_files = []
        for file in os.listdir(results_dir):
            if file.endswith(('.jpg', '.png')) and ('val_batch' in file or 'train_batch' in file):
                result_files.append(os.path.join(results_dir, file))

        # Отображаем первые 3 изображения с контурами
        for i, img_path in enumerate(result_files[:3]):
            img = Image.open(img_path)
            plt.figure(figsize=(12, 8))
            plt.imshow(img)
            plt.title(f'{model_name} - Результат {i+1}')
            plt.axis('off')
            plt.tight_layout()
            plt.show()

        return extract_yolo_metrics(model_name)

    elif model_type == 'sam':
        images, masks = test_data
        device = next(model.parameters()).device

        for i in range(min(num_samples, len(images))):
            img_path = images[i]
            mask_path = masks[i]

            # Загрузка данных
            img = np.array(Image.open(img_path))
            true_mask = np.array(Image.open(mask_path))

            # Предсказание SAM 2
            model.eval()
            if len(img.shape) == 2:
                img = np.stack([img, img, img], axis=-1)
            img_resized = cv2.resize(img, (1024, 1024), interpolation=cv2.INTER_LINEAR)
            img_resized = img_resized.astype(np.float32) / 255.0
            img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).unsqueeze(0).to(device)

            with torch.no_grad():
                image_embedding = model.image_encoder(img_tensor)
                sparse_embeddings, dense_embeddings = model.prompt_encoder(
                    points=None, boxes=None, masks=None
                )
                low_res_masks, _ = model.mask_decoder(
                    image_embeddings=image_embedding,
                    image_pe=model.prompt_encoder.get_dense_pe(),
                    sparse_prompt_embeddings=sparse_embeddings,
                    dense_prompt_embeddings=dense_embeddings,
                    multimask_output=False,
                )
                pred_mask = (torch.sigmoid(low_res_masks) > 0.5).float().squeeze().cpu().numpy()

            # Изменяем размер предсказания к размеру исходной маски
            pred_mask_resized = cv2.resize(pred_mask, (true_mask.shape[1], true_mask.shape[0]),
                                         interpolation=cv2.INTER_NEAREST)

            # Вычисление метрик
            dice = dc(pred_mask_resized, true_mask)
            iou = jc(pred_mask_resized, true_mask)
            intersection = np.logical_and(pred_mask_resized, true_mask).sum()
            precision = intersection / (pred_mask_resized.sum() + 1e-8)
            recall = intersection / (true_mask.sum() + 1e-8)

            dices.append(dice)
            ious.append(iou)
            precisions.append(precision)
            recalls.append(recall)

            # Визуализация первых 3 примеров с контурами
            if i < 3:
                # Подготовка контуров
                true_mask_8uc1 = (true_mask * 255).astype(np.uint8)
                pred_mask_8uc1 = (pred_mask_resized * 255).astype(np.uint8)

                contours_true, _ = cv2.findContours(true_mask_8uc1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                contours_pred, _ = cv2.findContours(pred_mask_8uc1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                plt.figure(figsize=(12, 4))
                plt.subplot(1, 3, 1)
                if len(img.shape) == 3:
                    plt.imshow(img[:,:,0], cmap='gray')
                else:
                    plt.imshow(img, cmap='gray')
                plt.title('Исходное КТ')
                plt.axis('off')

                plt.subplot(1, 3, 2)
                plt.imshow(true_mask, cmap='jet')
                for contour in contours_true:
                    plt.plot(contour[:, 0, 0], contour[:, 0, 1], 'b', linewidth=2)
                plt.title('Истинная маска')
                plt.axis('off')

                plt.subplot(1, 3, 3)
                plt.imshow(pred_mask_resized, cmap='jet')
                for contour in contours_pred:
                    plt.plot(contour[:, 0, 0], contour[:, 0, 1], 'r', linewidth=2)
                plt.title(f'Предсказание\nDice: {dice:.3f}')
                plt.axis('off')

                plt.suptitle(f'{model_name} - Пример {i+1}')
                plt.tight_layout()
                plt.show()

    # Статистика для U-Net и SAM
    metrics = {
        'dice_mean': np.mean(dices) if dices else 0,
        'dice_std': np.std(dices) if dices else 0,
        'iou_mean': np.mean(ious) if ious else 0,
        'iou_std': np.std(ious) if ious else 0,
        'precision_mean': np.mean(precisions) if precisions else 0,
        'precision_std': np.std(precisions) if precisions else 0,
        'recall_mean': np.mean(recalls) if recalls else 0,
        'recall_std': np.std(recalls) if recalls else 0,
        'num_samples': len(dices)
    }

    print(f"Результаты оценки на {metrics['num_samples']} samples:")
    print(f"Dice Coefficient: {metrics['dice_mean']:.4f} ± {metrics['dice_std']:.4f}")
    print(f"Jaccard Index (IoU): {metrics['iou_mean']:.4f} ± {metrics['iou_std']:.4f}")
    print(f"Precision: {metrics['precision_mean']:.4f} ± {metrics['precision_std']:.4f}")
    print(f"Recall: {metrics['recall_mean']:.4f} ± {metrics['recall_std']:.4f}")

    return metrics

def visualize_training_history(history, model_name, history_type='unet'):
    """
    Визуализирует историю обучения
    """
    plt.figure(figsize=(15, 5))
    if history_type == 'unet':
        # Loss
        plt.subplot(1, 3, 1)
        plt.plot(history.history['loss'], label='train_loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.title(f'{model_name} Loss')
        plt.legend()

        # Dice
        plt.subplot(1, 3, 2)
        plt.plot(history.history['dice_coefficient'], label='train_dice')
        plt.plot(history.history['val_dice_coefficient'], label='val_dice')
        plt.title(f'{model_name} Dice Coefficient')
        plt.legend()

        # IoU
        plt.subplot(1, 3, 3)
        plt.plot(history.history['jaccard_index'], label='train_iou')
        plt.plot(history.history['val_jaccard_index'], label='val_iou')
        plt.title(f'{model_name} IoU')
        plt.legend()
    elif history_type == 'sam':
        # Для SAM
        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='train_loss')
        plt.plot(history['val_loss'], label='val_loss')
        plt.title(f'{model_name} Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history['train_dice'], label='train_dice')
        plt.plot(history['val_dice'], label='val_dice')
        plt.title(f'{model_name} Dice Coefficient')
        plt.legend()

    plt.tight_layout()
    plt.savefig(f'/content/{model_name}_training_history.png')
    plt.show()

def extract_yolo_metrics(model_name):
    """
    Извлекает метрики YOLO из CSV файла
    """
    import pandas as pd

    results_base_dir = '/content/runs/segment'
    if not os.path.exists(results_base_dir):
        return {'dice_mean': 0.0, 'iou_mean': 0.0, 'precision_mean': 0.0, 'recall_mean': 0.0, 'num_samples': 0}

    all_folders = [f for f in os.listdir(results_base_dir)
                   if os.path.isdir(os.path.join(results_base_dir, f)) and 'yolo11' in f]
    if not all_folders:
        return {'dice_mean': 0.0, 'iou_mean': 0.0, 'precision_mean': 0.0, 'recall_mean': 0.0, 'num_samples': 0}

    all_folders.sort()
    latest_folder = all_folders[-1]
    csv_path = os.path.join(results_base_dir, latest_folder, 'results.csv')

    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        last_row = df.iloc[-1]
        metrics = {
            'dice_mean': last_row.get('metrics/mAP50(M)', 0.0),
            'iou_mean': last_row.get('metrics/mAP50-95(M)', 0.0),
            'precision_mean': last_row.get('metrics/precision(M)', 0.0),
            'recall_mean': last_row.get('metrics/recall(M)', 0.0),
            'num_samples': 20
        }
        return metrics
    else:
        return {'dice_mean': 0.0, 'iou_mean': 0.0, 'precision_mean': 0.0, 'recall_mean': 0.0, 'num_samples': 0}

#ЗАПУСК ЭКСПЕРИМЕНТОВ

In [ ]:
print("Подготовка данных для всех моделей")

# Создаем рабочую конфигурацию из базовой
config = BASE_CONFIG.copy()

# Подготовка данных для U-Net
print("\nПодготовка данных для U-Net")
train_gen, val_gen, steps_per_epoch, val_steps = prepare_unet_data(X_df, y_df, config)

# Подготовка данных для YOLO
print("\nПодготовка данных для YOLO")
yolo_config_path, yolo_train_size, yolo_val_size = prepare_yolo_data(X_df, y_df)

# Подготовка данных для SAM
print("\nПодготовка данных для SAM")
sam_train_images, sam_train_masks, sam_val_images, sam_val_masks = prepare_sam_data(X_df, y_df)
print("Все данные подготовлены")

# Словарь для хранения результатов
all_results = {}
best_models = {}

# Эксперимент 1: U-Net
print("\n" + "="*60)
print("ЭКСПЕРИМЕНТ 1: U-Net")
print("="*60)
model_unet = ModelFactory.create_unet((config['img_size'], config['img_size'], 1), neurons=32, name='unet_liver')
history_unet = train_unet_model(model_unet, train_gen, val_gen, steps_per_epoch, val_steps, config, 'unet')
visualize_training_history(history_unet, 'U-Net', 'unet')

# Загружаем лучшую версию модели для оценки
print("Загружаем лучшую версию U-Net для оценки")
best_unet = tf.keras.models.load_model('/content/unet_best.h5', custom_objects={
    'combined_loss': combined_loss,
    'dice_coefficient': dice_coefficient,
    'jaccard_index': jaccard_index,
    'precision_metric': precision_metric,
    'recall_metric': recall_metric
})
eval_metrics_unet = evaluate_model_with_contours(best_unet, val_gen, 'unet', 'U-Net (best)', 10)
best_models['U-Net'] = best_unet
all_results['U-Net'] = {
    'eval_metrics': eval_metrics_unet,
    'model_params': model_unet.count_params() / 1e6
}

# Эксперимент 2: YOLO11n-seg
print("\n" + "="*60)
print("ЭКСПЕРИМЕНТ 2: YOLO11n-seg")
print("="*60)
model_yolo11n = ModelFactory.create_yolo11n_seg()
history_yolo11n = train_yolo_model(model_yolo11n, yolo_config_path, config, 'yolo11n')

# Загружаем лучшую версию YOLO
print("Загружаем лучшую версию YOLO11n-seg для оценки")
best_yolo11n = YOLO('/content/yolo11n_best.pt')
eval_metrics_yolo11n = evaluate_model_with_contours(best_yolo11n, None, 'yolo', 'YOLO11n-seg (best)', 10)
best_models['YOLO11n-seg'] = best_yolo11n
all_results['YOLO11n-seg'] = {
    'eval_metrics': eval_metrics_yolo11n,
    'model_params': sum(p.numel() for p in model_yolo11n.model.parameters()) / 1e6
}

# Эксперимент 3: YOLO11s-seg
print("\n" + "="*60)
print("ЭКСПЕРИМЕНТ 3: YOLO11s-seg")
print("="*60)
model_yolo11s = ModelFactory.create_yolo11s_seg()
history_yolo11s = train_yolo_model(model_yolo11s, yolo_config_path, config, 'yolo11s')

# Загружаем лучшую версию YOLO
print("Загружаем лучшую версию YOLO11s-seg для оценки")
best_yolo11s = YOLO('/content/yolo11s_best.pt')
eval_metrics_yolo11s = evaluate_model_with_contours(best_yolo11s, None, 'yolo', 'YOLO11s-seg (best)', 10)
best_models['YOLO11s-seg'] = best_yolo11s
all_results['YOLO11s-seg'] = {
    'eval_metrics': eval_metrics_yolo11s,
    'model_params': sum(p.numel() for p in model_yolo11s.model.parameters()) / 1e6
}

In [ ]:
def visualize_yolo_training_history(model_name, title_suffix=''):
    """
    Визуализирует историю обучения YOLO модели на основе results.csv
    """
    import pandas as pd
    import matplotlib.pyplot as plt

    results_base_dir = '/content/runs/segment'

    all_folders = [f for f in os.listdir(results_base_dir)
                  if os.path.isdir(os.path.join(results_base_dir, f)) and model_name in f]

    all_folders.sort()
    latest_folder = all_folders[-1]
    results_dir = os.path.join(results_base_dir, latest_folder)

    print(f"Загружаем данные обучения из: {latest_folder}")

    # Путь к CSV файлу с результатами
    csv_path = os.path.join(results_dir, 'results.csv')

    if not os.path.exists(csv_path):
        print(f"Файл результатов не найден: {csv_path}")
        return

    # Загружаем данные
    df = pd.read_csv(csv_path)

    # Создаем графики
    plt.figure(figsize=(15, 10))

    # Loss функции
    plt.subplot(2, 2, 1)
    plt.plot(df['epoch'], df['train/box_loss'], label='Box Loss (train)', alpha=0.7)
    plt.plot(df['epoch'], df['train/cls_loss'], label='Cls Loss (train)', alpha=0.7)
    plt.plot(df['epoch'], df['train/dfl_loss'], label='DFL Loss (train)', alpha=0.7)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'{model_name} {title_suffix} - Training Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Валидационные loss
    plt.subplot(2, 2, 2)
    plt.plot(df['epoch'], df['val/box_loss'], label='Box Loss (val)', alpha=0.7)
    plt.plot(df['epoch'], df['val/cls_loss'], label='Cls Loss (val)', alpha=0.7)
    plt.plot(df['epoch'], df['val/dfl_loss'], label='DFL Loss (val)', alpha=0.7)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'{model_name} {title_suffix} - Validation Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Метрики сегментации
    plt.subplot(2, 2, 3)
    plt.plot(df['epoch'], df['metrics/precision(M)'], label='Precision', alpha=0.7)
    plt.plot(df['epoch'], df['metrics/recall(M)'], label='Recall', alpha=0.7)
    plt.plot(df['epoch'], df['metrics/mAP50(M)'], label='mAP@0.5', alpha=0.7)
    plt.plot(df['epoch'], df['metrics/mAP50-95(M)'], label='mAP@0.5-0.95', alpha=0.7)
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.title(f'{model_name} {title_suffix} - Segmentation Metrics')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Скорость обучения
    plt.subplot(2, 2, 4)
    plt.plot(df['epoch'], df['lr/pg0'], label='LR pg0', alpha=0.7)
    plt.plot(df['epoch'], df['lr/pg1'], label='LR pg1', alpha=0.7)
    plt.plot(df['epoch'], df['lr/pg2'], label='LR pg2', alpha=0.7)
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.title(f'{model_name} {title_suffix} - Learning Rate Schedule')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'/content/{model_name}_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

    # График mAP и Precision-Recall
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(df['epoch'], df['metrics/mAP50(M)'], 'b-', label='mAP@0.5', linewidth=2)
    plt.plot(df['epoch'], df['metrics/mAP50-95(M)'], 'g-', label='mAP@0.5-0.95', linewidth=2)
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.title(f'{model_name} {title_suffix} - mAP Evolution')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(df['epoch'], df['metrics/precision(M)'], 'r-', label='Precision', linewidth=2)
    plt.plot(df['epoch'], df['metrics/recall(M)'], 'c-', label='Recall', linewidth=2)
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.title(f'{model_name} {title_suffix} - Precision vs Recall')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'/content/{model_name}_metrics_evolution.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Выводим лучшие результаты
    best_epoch = df['metrics/mAP50(M)'].idxmax()
    print(f"\nЛучшие результаты для {model_name}:")
    print(f"Эпоха: {best_epoch + 1}")
    print(f"mAP@0.5: {df['metrics/mAP50(M)'].iloc[best_epoch]:.4f}")
    print(f"mAP@0.5-0.95: {df['metrics/mAP50-95(M)'].iloc[best_epoch]:.4f}")
    print(f"Precision: {df['metrics/precision(M)'].iloc[best_epoch]:.4f}")
    print(f"Recall: {df['metrics/recall(M)'].iloc[best_epoch]:.4f}")

    return df

In [ ]:
# Визуализация истории обучения для YOLO11n-seg
print("\nВизуализация истории обучения YOLO11n-seg")
yolo11n_metrics = visualize_yolo_training_history('yolo11n', '(nano version)')

# Загружаем лучшую версию YOLO
print("Загружаем лучшую версию YOLO11n-seg для оценки")
best_yolo11n = YOLO('/content/yolo11n_best.pt')
eval_metrics_yolo11n = evaluate_model_with_contours(best_yolo11n, None, 'yolo', 'YOLO11n-seg (best)', 10)
best_models['YOLO11n-seg'] = best_yolo11n
all_results['YOLO11n-seg'] = {
    'eval_metrics': eval_metrics_yolo11n,
    'model_params': sum(p.numel() for p in model_yolo11n.model.parameters()) / 1e6
}

# Визуализация истории обучения для YOLO11s-seg
print("\nВизуализация истории обучения YOLO11s-seg")
yolo11s_metrics = visualize_yolo_training_history('yolo11s', '(small version)')

# Загружаем лучшую версию YOLO
print("Загружаем лучшую версию YOLO11s-seg для оценки")
best_yolo11s = YOLO('/content/yolo11s_best.pt')
eval_metrics_yolo11s = evaluate_model_with_contours(best_yolo11s, None, 'yolo', 'YOLO11s-seg (best)', 10)
best_models['YOLO11s-seg'] = best_yolo11s
all_results['YOLO11s-seg'] = {
    'eval_metrics': eval_metrics_yolo11s,
    'model_params': sum(p.numel() for p in model_yolo11s.model.parameters()) / 1e6
}

In [ ]:
# Эксперимент 4: SAM 2
print("\n" + "="*60)
print("ЭКСПЕРИМЕНТ 4: SAM 2")
print("="*60)
model_sam = ModelFactory.create_sam2_model()
history_sam = train_sam_model(model_sam, (sam_train_images, sam_train_masks), (sam_val_images, sam_val_masks), config, 'sam2')
visualize_training_history(history_sam, 'SAM 2', 'sam')

# Загружаем лучшую версию SAM 2
print("Загружаем лучшую версию SAM 2 для оценки")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_sam = ModelFactory.create_sam2_model()
checkpoint = torch.load('/content/sam2_best.pth')
best_sam.load_state_dict(checkpoint['model_state_dict'])
best_sam = best_sam.to(device)
best_sam.eval()

eval_metrics_sam = evaluate_model_with_contours(best_sam, (sam_val_images, sam_val_masks), 'sam', 'SAM 2 (best)', 10)
best_models['SAM 2'] = best_sam
all_results['SAM 2'] = {
    'eval_metrics': eval_metrics_sam,
    'model_params': sum(p.numel() for p in best_sam.parameters()) / 1e6
}

#ВЛИЯНИЕ LOSS ФУНКЦИЙ НА U-NET И YOLO



In [ ]:
print("\n" + "="*80)
print("ВЛИЯНИЕ LOSS ФУНКЦИЙ")
print("="*80)

import re

def clean_model_name(name):
    """Очищает имя модели от недопустимых символов"""
    # Заменяем все недопустимые символы на подчеркивания
    return re.sub(r'[^\w\-.]', '_', name.lower().replace(' ', '_'))

# Различные loss функции для тестирования - ИСПРАВЛЕННЫЕ ВЕРСИИ
def focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    """Focal loss для балансировки классов"""
    bce = tf.keras.losses.BinaryCrossentropy(reduction='none')(y_true, y_pred)
    pt = tf.exp(-bce)
    focal_loss = alpha * tf.pow(1. - pt, gamma) * bce
    return tf.reduce_mean(focal_loss)

def tversky_loss(y_true, y_pred, alpha=0.7, beta=0.3, smooth=1e-7):
    """Tversky loss для фокуса на границах"""
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    true_pos = tf.reduce_sum(y_true_f * y_pred_f)
    false_neg = tf.reduce_sum(y_true_f * (1 - y_pred_f))
    false_pos = tf.reduce_sum((1 - y_true_f) * y_pred_f)
    return 1 - (true_pos + smooth) / (true_pos + alpha * false_neg + beta * false_pos + smooth)

def dice_loss_fixed(y_true, y_pred, smooth=1e-7):
    """
    Исправленная Dice loss функция с дифференцируемыми операциями
    """
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice

def combined_loss_fixed(y_true, y_pred, alpha=0.5):
    """
    Исправленная комбинированная loss функция
    """
    dice = dice_loss_fixed(y_true, y_pred)
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    return alpha * dice + (1 - alpha) * bce

# Конфигурации loss функций
loss_configs = [
    ('BCE Only', lambda y_true, y_pred: tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)),
    ('Dice Only', dice_loss_fixed),
    ('Combined Dice BCE', combined_loss_fixed),
    ('Focal Loss', focal_loss),
    ('Tversky Loss', tversky_loss)
]

# Настройки для быстрых экспериментов
config_ablation = BASE_CONFIG.copy()
config_ablation['epochs'] = 5
config_ablation['img_size'] = 256
config_ablation['batch_size'] = 4

# Подготовка данных
train_gen_abl, val_gen_abl, steps_per_epoch_abl, val_steps_abl = create_data_generators(X_df, y_df, config_ablation)

results_by_loss = {}

# Тестируем только на U-Net и одной YOLO модели
print("\n=== ЭКСПЕРИМЕНТЫ С U-NET ===")
for loss_name, loss_func in loss_configs:
    print(f"\n{'-'*40}")
    print(f"U-Net с loss: {loss_name}")
    print(f"{'-'*40}")

    clean_name = clean_model_name(loss_name)

    try:
        model_unet_abl = ModelFactory.create_unet((config_ablation['img_size'], config_ablation['img_size'], 1),
                                                 neurons=16, name=f'unet_{clean_name}')

        model_unet_abl.compile(
            optimizer=Adam(learning_rate=config_ablation['learning_rate']),
            loss=loss_func,
            metrics=[dice_coefficient, jaccard_index, precision_metric, recall_metric]
        )

        # Обучение
        history_unet_abl = model_unet_abl.fit(
            train_gen_abl,
            validation_data=val_gen_abl,
            steps_per_epoch=steps_per_epoch_abl,
            validation_steps=val_steps_abl,
            epochs=config_ablation['epochs'],
            verbose=1
        )

        # Оценка с визуализацией контуров
        eval_metrics = evaluate_model_with_contours(model_unet_abl, val_gen_abl, 'unet', f'U-Net {loss_name}', 3)

        results_by_loss[f'U-Net {loss_name}'] = {
            'dice': eval_metrics['dice_mean'],
            'iou': eval_metrics['iou_mean'],
            'precision': eval_metrics['precision_mean'],
            'recall': eval_metrics['recall_mean'],
            'history': history_unet_abl.history
        }

        # Сохраняем модель
        save_path = f'/content/unet_{clean_name}.keras'
        model_unet_abl.save(save_path)
        print(f"Модель сохранена в: {save_path}")

    except Exception as e:
        print(f"Ошибка при обучении U-Net с loss '{loss_name}': {str(e)}")
        continue

print("\n=== ЭКСПЕРИМЕНТЫ С YOLO11n-seg ===")
yolo_variants = {
    'Standard': {},
    'High_Recall': {
        'box': 7.5,      # вес box loss
        'cls': 0.5,      # вес cls loss
        'dfl': 1.5       # вес dfl loss
    },
    'High_Precision': {
        'box': 0.05,     # уменьшаем вес box loss для лучшей precision
        'cls': 5.0,      # увеличиваем вес cls loss
        'dfl': 1.0
    }
}

for variant_name, params in yolo_variants.items():
    print(f"\n{'-'*40}")
    print(f"YOLO11n-seg: {variant_name}")
    print(f"{'-'*40}")

    output_dir_temp = f'/content/yolo_{variant_name.lower()}'
    if os.path.exists(output_dir_temp):
        shutil.rmtree(output_dir_temp)
    os.makedirs(output_dir_temp, exist_ok=True)

    # Копируем данные YOLO
    if os.path.exists('/content/yolo_dataset'):
        shutil.copytree('/content/yolo_dataset', output_dir_temp, dirs_exist_ok=True)
    else:
        print(f"Предупреждение: директория /content/yolo_dataset не существует. Создаем базовую структуру.")
        # Создаем базовую структуру YOLO
        for split in ['train', 'val']:
            os.makedirs(os.path.join(output_dir_temp, 'images', split), exist_ok=True)
            os.makedirs(os.path.join(output_dir_temp, 'labels', split), exist_ok=True)

    temp_config_path = os.path.join(output_dir_temp, 'data.yaml')

    # Создаем конфигурационный файл если его нет
    if not os.path.exists(temp_config_path):
        yolo_config = {
            'path': output_dir_temp,
            'train': 'images/train',
            'val': 'images/val',
            'names': {0: 'liver'},
            'nc': 1
        }
        with open(temp_config_path, 'w') as f:
            yaml.dump(yolo_config, f)
        print(f"Создан конфигурационный файл: {temp_config_path}")

    try:
        model_yolo_abl = YOLO('yolo11n-seg.pt')

        # Обучение с правильными параметрами
        train_args = {
            'data': temp_config_path,
            'epochs': config_ablation['epochs'],
            'batch': config_ablation['batch_size'],
            'imgsz': config_ablation['img_size'],
            'name': f'yolo11n_{variant_name.lower()}',
            'patience': 3,
            'device': 'cpu',
            'verbose': False
        }

        # Добавляем правильные параметры обучения
        if params:
            # Параметры YOLO называются именно так согласно документации
            for k, v in params.items():
                train_args[k] = v

        print(f"Параметры обучения: {train_args}")

        results_yolo_abl = model_yolo_abl.train(**train_args)

        # Оценка
        eval_metrics = extract_yolo_metrics(f'yolo11n_{variant_name.lower()}')

        results_by_loss[f'YOLO {variant_name}'] = {
            'dice': eval_metrics['dice_mean'],
            'iou': eval_metrics['iou_mean'],
            'precision': eval_metrics['precision_mean'],
            'recall': eval_metrics['recall_mean']
        }

        # Сохраняем модель
        save_path = f'/content/yolo11n_{variant_name.lower()}.pt'
        model_yolo_abl.save(save_path)
        print(f"Модель сохранена в: {save_path}")

    except Exception as e:
        print(f"Ошибка при обучении YOLO11n-seg с вариантом '{variant_name}': {str(e)}")
        continue

# Визуализация результатов (остается без изменений)
plt.figure(figsize=(15, 8))

# Сравнение Dice для всех моделей
plt.subplot(2, 2, 1)
dice_scores = []
model_names = []
colors = []
for name, results in results_by_loss.items():
    dice_scores.append(results['dice'])
    model_names.append(name)
    colors.append('blue' if 'U-Net' in name else 'green')

x = np.arange(len(model_names))
plt.bar(x, dice_scores, alpha=0.7, color=colors)
plt.xlabel('Модели и Loss функции')
plt.ylabel('Dice Coefficient')
plt.title('Сравнение Dice по разным loss функциям')
plt.xticks(x, [name.replace('U-Net ', '').replace('YOLO ', '') for name in model_names], rotation=45)
plt.grid(True, alpha=0.3)

# Добавляем значения на столбцы
for i, v in enumerate(dice_scores):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center')

# Сравнение IoU
plt.subplot(2, 2, 2)
iou_scores = [results['iou'] for results in results_by_loss.values()]
plt.bar(x, iou_scores, alpha=0.7, color=colors)
plt.xlabel('Модели и Loss функции')
plt.ylabel('IoU')
plt.title('Сравнение IoU по разным loss функциям')
plt.xticks(x, [name.replace('U-Net ', '').replace('YOLO ', '') for name in model_names], rotation=45)
plt.grid(True, alpha=0.3)

# Сравнение Precision-Recall для U-Net
plt.subplot(2, 2, 3)
unet_results = {k: v for k, v in results_by_loss.items() if 'U-Net' in k}
if unet_results:
    unet_names = list(unet_results.keys())
    precisions = [results['precision'] for results in unet_results.values()]
    recalls = [results['recall'] for results in unet_results.values()]

    x_unet = np.arange(len(unet_names))
    width = 0.35

    plt.bar(x_unet - width/2, precisions, width, label='Precision', alpha=0.7)
    plt.bar(x_unet + width/2, recalls, width, label='Recall', alpha=0.7)
    plt.xlabel('Loss функции для U-Net')
    plt.ylabel('Score')
    plt.title('Precision vs Recall для разных loss функций U-Net')
    plt.xticks(x_unet, [name.replace('U-Net ', '') for name in unet_names], rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/content/ablation_loss_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Вывод результатов
print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ ПО LOSS ФУНКЦИЯМ")
print("="*60)

# Лучшие результаты для U-Net
unet_results = {k: v for k, v in results_by_loss.items() if 'U-Net' in k}
if unet_results:
    print("\nЛУЧШИЕ LOSS ФУНКЦИИ ДЛЯ U-NET:")
    unet_results_sorted = sorted([(k, v['dice']) for k, v in unet_results.items()],
                                 key=lambda x: x[1], reverse=True)
    for model_name, dice_score in unet_results_sorted:
        print(f"{model_name}: Dice = {dice_score:.4f}")

    best_unet_loss = unet_results_sorted[0][0].replace('U-Net ', '')
else:
    print("\nНет успешных результатов для U-Net")
    best_unet_loss = "Combined Dice BCE"

# Лучшие результаты для YOLO
yolo_results = {k: v for k, v in results_by_loss.items() if 'YOLO' in k}
if yolo_results:
    print("\nЛУЧШИЕ ПАРАМЕТРЫ ДЛЯ YOLO11n-seg:")
    yolo_results_sorted = sorted([(k, v['dice']) for k, v in yolo_results.items()],
                                 key=lambda x: x[1], reverse=True)
    for model_name, dice_score in yolo_results_sorted:
        print(f"{model_name}: Dice = {dice_score:.4f}")

    best_yolo_variant = yolo_results_sorted[0][0].replace('YOLO ', '')
else:
    print("\nНет успешных результатов для YOLO")
    best_yolo_variant = "Standard"

print("\nРЕКОМЕНДАЦИИ:")
print(f"Для U-Net рекомендуется использовать: {best_unet_loss}")
print(f"Для YOLO11n-seg рекомендуется использовать: {best_yolo_variant}")

In [ ]:
print("\n" + "="*80)
print("ВЛИЯНИЕ LOSS ФУНКЦИЙ")
print("="*80)

import re

def clean_name(name):
    """Очищает имя от недопустимых символов для TensorFlow/Keras"""
    return re.sub(r'[^a-zA-Z0-9_\-]', '_', name.lower().replace(' ', '_'))

# Различные loss функции для тестирования
def focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    """Focal loss для балансировки классов"""
    bce = tf.keras.losses.BinaryCrossentropy(reduction='none')(y_true, y_pred)
    pt = tf.exp(-bce)
    focal_loss = alpha * tf.pow(1. - pt, gamma) * bce
    return tf.reduce_mean(focal_loss)

def tversky_loss(y_true, y_pred, alpha=0.7, beta=0.3, smooth=1e-7):
    """Tversky loss для фокуса на границах"""
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    true_pos = tf.reduce_sum(y_true_f * y_pred_f)
    false_neg = tf.reduce_sum(y_true_f * (1 - y_pred_f))
    false_pos = tf.reduce_sum((1 - y_true_f) * y_pred_f)
    return 1 - (true_pos + smooth) / (true_pos + alpha * false_neg + beta * false_pos + smooth)

def dice_loss_fixed(y_true, y_pred, smooth=1e-7):
    """Исправленная Dice loss функция"""
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice

def combined_loss_fixed(y_true, y_pred, alpha=0.5):
    """Исправленная комбинированная loss функция"""
    dice = dice_loss_fixed(y_true, y_pred)
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    return alpha * dice + (1 - alpha) * bce

# Конфигурации loss функций с чистыми именами
loss_configs = [
    ('BCE_Only', lambda y_true, y_pred: tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)),
    ('Dice_Only', dice_loss_fixed),
    ('Combined_Dice_BCE', combined_loss_fixed),
    ('Focal_Loss', focal_loss),
    ('Tversky_Loss', tversky_loss)
]

# Настройки для быстрых экспериментов
config_ablation = BASE_CONFIG.copy()
config_ablation['epochs'] = 5
config_ablation['img_size'] = 256
config_ablation['batch_size'] = 4

# Подготовка данных
train_gen_abl, val_gen_abl, steps_per_epoch_abl, val_steps_abl = create_data_generators(X_df, y_df, config_ablation)

results_by_loss = {}

# Тестируем только на U-Net
print("\n=== ЭКСПЕРИМЕНТЫ С U-NET ===")
for loss_name_clean, loss_func in loss_configs:
    original_name = loss_name_clean.replace('_', ' ')
    print(f"\n{'-'*40}")
    print(f"U-Net с loss: {original_name}")
    print(f"{'-'*40}")

    try:
        model_unet_abl = ModelFactory.create_unet((config_ablation['img_size'], config_ablation['img_size'], 1),
                                                 neurons=16, name=f'unet_{loss_name_clean}')

        model_unet_abl.compile(
            optimizer=Adam(learning_rate=config_ablation['learning_rate']),
            loss=loss_func,
            metrics=[dice_coefficient, jaccard_index, precision_metric, recall_metric]
        )

        # Обучение
        history_unet_abl = model_unet_abl.fit(
            train_gen_abl,
            validation_data=val_gen_abl,
            steps_per_epoch=steps_per_epoch_abl,
            validation_steps=val_steps_abl,
            epochs=config_ablation['epochs'],
            verbose=1
        )

        # Оценка
        eval_metrics = evaluate_model(model_unet_abl, val_gen_abl, 'unet', f'U-Net {original_name}', 3)

        results_by_loss[f'U-Net {original_name}'] = {
            'dice': eval_metrics['dice_mean'],
            'iou': eval_metrics['iou_mean'],
            'precision': eval_metrics['precision_mean'],
            'recall': eval_metrics['recall_mean'],
            'history': history_unet_abl.history
        }

        # Сохраняем модель
        save_path = f'/content/unet_{loss_name_clean}.h5'
        model_unet_abl.save(save_path)
        print(f"Модель сохранена в: {save_path}")

    except Exception as e:
        print(f"Ошибка при обучении U-Net с loss '{original_name}': {str(e)}")
        continue

# Параметры YOLO без спецсимволов в именах
yolo_variants = {
    'Standard': {},
    'High_Recall': {
        'box': 7.5,      # вес box loss
        'cls': 0.1,      # уменьшаем вес cls loss для лучшего recall
        'dfl': 1.5       # вес dfl loss
    },
    'High_Precision': {
        'box': 0.1,      # уменьшаем вес box loss
        'cls': 5.0,      # увеличиваем вес cls loss для лучшей precision
        'dfl': 1.0
    }
}

print("\n=== ЭКСПЕРИМЕНТЫ С YOLO11n-seg ===")
for variant_clean, params in yolo_variants.items():
    variant_name = variant_clean.replace('_', ' ')
    print(f"\n{'-'*40}")
    print(f"YOLO11n-seg: {variant_name}")
    print(f"{'-'*40}")

    output_dir_temp = f'/content/yolo_{variant_clean}'
    if os.path.exists(output_dir_temp):
        shutil.rmtree(output_dir_temp)
    os.makedirs(output_dir_temp, exist_ok=True)

    shutil.copytree('/content/yolo_dataset', output_dir_temp, dirs_exist_ok=True)
    temp_config_path = os.path.join(output_dir_temp, 'data.yaml')

    try:
        model_yolo_abl = YOLO('yolo11n-seg.pt')

        train_args = {
            'data': temp_config_path,
            'epochs': config_ablation['epochs'],
            'batch': config_ablation['batch_size'],
            'imgsz': config_ablation['img_size'],
            'name': f'yolo11n_{variant_clean}',
            'patience': 3,
            'device': 'cpu',
            'verbose': False
        }

        if params:
            for k, v in params.items():
                train_args[k] = v

        print(f"Параметры обучения: {train_args}")

        results_yolo_abl = model_yolo_abl.train(**train_args)

        eval_metrics = extract_yolo_metrics(f'yolo11n_{variant_clean}')

        results_by_loss[f'YOLO {variant_name}'] = {
            'dice': eval_metrics['dice_mean'],
            'iou': eval_metrics['iou_mean'],
            'precision': eval_metrics['precision_mean'],
            'recall': eval_metrics['recall_mean']
        }

        save_path = f'/content/yolo11n_{variant_clean}.pt'
        model_yolo_abl.save(save_path)
        print(f"Модель сохранена в: {save_path}")

    except Exception as e:
        print(f"Ошибка при обучении YOLO11n-seg с вариантом '{variant_name}': {str(e)}")
        continue

# Визуализация результатов
if results_by_loss:
    plt.figure(figsize=(15, 8))

    # Сравнение Dice для всех моделей
    plt.subplot(2, 2, 1)
    dice_scores = [results['dice'] for results in results_by_loss.values()]
    model_names = list(results_by_loss.keys())
    x = np.arange(len(model_names))

    plt.bar(x, dice_scores, alpha=0.7, color=['blue' if 'U-Net' in name else 'green' for name in model_names])
    plt.xlabel('Модели и Loss функции')
    plt.ylabel('Dice Coefficient')
    plt.title('Сравнение Dice по разным loss функциям')
    plt.xticks(x, [name.replace('U-Net ', '').replace('YOLO ', '') for name in model_names], rotation=45)
    plt.grid(True, alpha=0.3)

    # Сравнение IoU
    plt.subplot(2, 2, 2)
    iou_scores = [results['iou'] for results in results_by_loss.values()]
    plt.bar(x, iou_scores, alpha=0.7, color=['blue' if 'U-Net' in name else 'green' for name in model_names])
    plt.xlabel('Модели и Loss функции')
    plt.ylabel('IoU')
    plt.title('Сравнение IoU по разным loss функциям')
    plt.xticks(x, [name.replace('U-Net ', '').replace('YOLO ', '') for name in model_names], rotation=45)
    plt.grid(True, alpha=0.3)

    # Сравнение Precision-Recall для U-Net
    plt.subplot(2, 2, 3)
    unet_results = {k: v for k, v in results_by_loss.items() if 'U-Net' in k}
    if unet_results:
        unet_names = list(unet_results.keys())
        precisions = [results['precision'] for results in unet_results.values()]
        recalls = [results['recall'] for results in unet_results.values()]

        x_unet = np.arange(len(unet_names))
        width = 0.35

        plt.bar(x_unet - width/2, precisions, width, label='Precision', alpha=0.7)
        plt.bar(x_unet + width/2, recalls, width, label='Recall', alpha=0.7)
        plt.xlabel('Loss функции для U-Net')
        plt.ylabel('Score')
        plt.title('Precision vs Recall для разных loss функций U-Net')
        plt.xticks(x_unet, [name.replace('U-Net ', '') for name in unet_names], rotation=45)
        plt.legend()
        plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('/content/ablation_loss_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

# Вывод результатов
print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ ПО LOSS ФУНКЦИЯМ")
print("="*60)

# Лучшие результаты для U-Net
unet_results = {k: v for k, v in results_by_loss.items() if 'U-Net' in k}
if unet_results:
    print("\nЛУЧШИЕ LOSS ФУНКЦИИ ДЛЯ U-NET:")
    unet_results_sorted = sorted([(k, v['dice']) for k, v in unet_results.items()],
                                 key=lambda x: x[1], reverse=True)
    for model_name, dice_score in unet_results_sorted:
        print(f"{model_name}: Dice = {dice_score:.4f}")

    best_unet_loss = unet_results_sorted[0][0].replace('U-Net ', '')
else:
    print("\nНет успешных результатов для U-Net")
    best_unet_loss = "Combined Dice BCE"

# Лучшие результаты для YOLO
yolo_results = {k: v for k, v in results_by_loss.items() if 'YOLO' in k}
if yolo_results:
    print("\nЛУЧШИЕ ПАРАМЕТРЫ ДЛЯ YOLO11n-seg:")
    yolo_results_sorted = sorted([(k, v['dice']) for k, v in yolo_results.items()],
                                 key=lambda x: x[1], reverse=True)
    for model_name, dice_score in yolo_results_sorted:
        print(f"{model_name}: Dice = {dice_score:.4f}")

    best_yolo_variant = yolo_results_sorted[0][0].replace('YOLO ', '')
else:
    print("\nНет успешных результатов для YOLO")
    best_yolo_variant = "Standard"

print("\nРЕКОМЕНДАЦИИ:")
print(f"Для U-Net рекомендуется использовать: {best_unet_loss}")
print(f"Для YOLO11n-seg рекомендуется использовать: {best_yolo_variant}")

#АНАЛИЗ РЕЗУЛЬТАТОВ

In [ ]:
print("\n" + "="*80)
print("АНАЛИЗ РЕЗУЛЬТАТОВ И ПРАКТИЧЕСКИЕ ВЫВОДЫ")
print("="*80)

# Сравнение улучшений
original_unet_dice = all_results['U-Net']['eval_metrics']['dice_mean']
best_unet_result = max([(k, v['dice']) for k, v in results_by_loss.items() if 'U-Net' in k],
                      key=lambda x: x[1])
best_unet_name, best_unet_dice = best_unet_result

original_yolo_dice = all_results['YOLO11s-seg']['eval_metrics']['dice_mean']
best_yolo_result = max([(k, v['dice']) for k, v in results_by_loss.items() if 'YOLO' in k],
                      key=lambda x: x[1])
best_yolo_name, best_yolo_dice = best_yolo_result

print(f"\nУЛУЧШЕНИЕ КАЧЕСТВА:")
print(f"U-Net: {original_unet_dice:.4f} → {best_unet_dice:.4f} (+{(best_unet_dice - original_unet_dice)*100:.1f}%)")
print(f"YOLO11s-seg: {original_yolo_dice:.4f} → {best_yolo_dice:.4f} (+{(best_yolo_dice - original_yolo_dice)*100:.1f}%)")

# Сохранение лучших моделей
print("\n" + "="*60)
print("СОХРАНЕНИЕ ЛУЧШИХ МОДЕЛЕЙ")
print("="*60)

best_unet_model = ModelFactory.create_unet((config_ablation['img_size'], config_ablation['img_size'], 1),
                                           neurons=16, name='best_unet_model')
best_unet_model.compile(
    optimizer=Adam(learning_rate=config_ablation['learning_rate']),
    loss=results_by_loss[best_unet_name]['history']['loss'][0],
    metrics=[dice_coefficient, jaccard_index, precision_metric, recall_metric]
)
best_unet_model.save(f'/content/best_unet_model.h5')
print(f"Лучшая U-Net модель сохранена: {best_unet_name}")

best_yolo_model = YOLO('yolo11s-seg.pt')
best_yolo_model.save(f'/content/best_yolo_model.pt')
print(f"Лучшая YOLO модель сохранена: {best_yolo_name}")

#СРАВНИТЕЛЬНЫЙ АНАЛИЗ ВСЕХ РЕЗУЛЬТАТОВ

In [ ]:
print("\n" + "="*80)
print("СРАВНИТЕЛЬНЫЙ АНАЛИЗ ВСЕХ РЕЗУЛЬТАТОВ")
print("="*80)

# Создание сравнительной таблицы
comparison_data = []

for model_name, results in all_results.items():
    eval_metrics = results['eval_metrics']
    comparison_data.append({
        'Model': model_name,
        'Type': 'Base',
        'Dice Coefficient': eval_metrics['dice_mean'],
        'Jaccard Index': eval_metrics['iou_mean'],
        'Precision': eval_metrics['precision_mean'],
        'Recall': eval_metrics['recall_mean'],
        'Parameters (M)': results['model_params']
    })

for model_name, results in results_by_loss.items():
    comparison_data.append({
        'Model': model_name,
        'Type': 'Ablation',
        'Dice Coefficient': results['dice'],
        'Jaccard Index': results['iou'],
        'Precision': results.get('precision', 0.0),
        'Recall': results.get('recall', 0.0),
        'Parameters (M)': 31.0 if 'U-Net' in model_name else 2.9
    })

results_df = pd.DataFrame(comparison_data)
print("\nВсе результаты:")
print(results_df.to_string(index=False, float_format='%.4f'))

# Визуализация результатов
plt.figure(figsize=(15, 12))

# Сравнение Dice по всем моделям
plt.subplot(2, 2, 1)
base_models = results_df[results_df['Type'] == 'Base']
ablation_models = results_df[results_df['Type'] == 'Ablation']

# Фильтруем только те базовые модели, для которых есть ablation версии
valid_base_models = []
valid_ablation_values = []

for base_model in base_models['Model']:
    # Ищем соответствующие ablation модели
    if 'U-Net' in base_model:
        ablation_model = ablation_models[ablation_models['Model'].str.contains('U-Net')]
    elif 'YOLO' in base_model:
        ablation_model = ablation_models[ablation_models['Model'].str.contains('YOLO')]
    else:
        ablation_model = pd.DataFrame()

    if not ablation_model.empty:
        valid_base_models.append(base_model)
        valid_ablation_values.append(ablation_model['Dice Coefficient'].mean())

x = np.arange(len(valid_base_models))
width = 0.35

if len(valid_base_models) > 0:
    base_dice_values = base_models[base_models['Model'].isin(valid_base_models)]['Dice Coefficient'].values
    plt.bar(x - width/2, base_dice_values, width, label='Base Models', alpha=0.7, color='blue')
    plt.bar(x + width/2, valid_ablation_values, width, label='Ablation Models', alpha=0.7, color='red')
    plt.xticks(x, valid_base_models, rotation=45)
else:
    plt.text(0.5, 0.5, 'Нет данных для сравнения Base и Ablation моделей',
             ha='center', va='center', transform=plt.gca().transAxes)

plt.xlabel('Models')
plt.ylabel('Dice Coefficient')
plt.title('Сравнение Dice: Base vs Ablation')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Соотношение параметров и качества
plt.subplot(2, 2, 2)
scatter = plt.scatter(results_df['Parameters (M)'], results_df['Dice Coefficient'],
                     s=100, c=results_df['Jaccard Index'], cmap='viridis', alpha=0.7)
plt.xlabel('Parameters (Millions)')
plt.ylabel('Dice Coefficient')
plt.title('Соотношение размера модели и качества')
plt.colorbar(scatter, label='IoU')
plt.grid(True, alpha=0.3)

for i, row in results_df.iterrows():
    plt.annotate(row['Model'].split()[0], (row['Parameters (M)'], row['Dice Coefficient']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

# Precision-Recall для лучших моделей
plt.subplot(2, 2, 3)
best_models_df = results_df.sort_values('Dice Coefficient', ascending=False).head(5)
plt.scatter(best_models_df['Precision'], best_models_df['Recall'],
           s=best_models_df['Dice Coefficient']*1000, alpha=0.7)

for i, row in best_models_df.iterrows():
    plt.annotate(row['Model'], (row['Precision'], row['Recall']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.xlabel('Precision')
plt.ylabel('Recall')
plt.title('Precision-Recall для лучших моделей')
plt.grid(True, alpha=0.3)

# Рейтинг моделей
plt.subplot(2, 2, 4)
composite_scores = (results_df['Dice Coefficient'] + results_df['Jaccard Index']) / 2
top_models = results_df.sort_values('Dice Coefficient', ascending=False).head(8)
plt.barh(top_models['Model'], composite_scores[top_models.index], color='skyblue')
plt.xlabel('Composite Score (Dice + IoU)/2')
plt.title('Рейтинг моделей по композитному score')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/content/final_comparison_all_models.png', dpi=300, bbox_inches='tight')
plt.show()

# Сохранение результатов
results_df.to_csv('/content/final_model_comparison_results.csv', index=False)
print("\nРезультаты сохранены в final_model_comparison_results.csv")

# Вывод лучших моделей
best_overall = results_df.loc[results_df['Dice Coefficient'].idxmax()]
print(f"\nЛучшая модель по Dice: {best_overall['Model']} (Dice: {best_overall['Dice Coefficient']:.4f})")
print(f"Тип: {best_overall['Type']}")
print(f"Jaccard Index: {best_overall['Jaccard Index']:.4f}")
print(f"Precision: {best_overall['Precision']:.4f}")
print(f"Recall: {best_overall['Recall']:.4f}")
